In [12]:
import pandas, copy, numpy,sys
from gumpy.genome import Genome
pandas.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2
numpy.set_printoptions(threshold=sys.maxsize)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

TEST_CASE_DIR = "tests/test-cases/"

In [3]:
reference=Genome(genbank_file="config/NC_004148.2.gbk",name="HMPV")

In [4]:
sample_01=copy.deepcopy(reference)
sample_01.apply_vcf_file(vcf_file=TEST_CASE_DIR+"01.vcf",ignore_status=True,ignore_filter=True,metadata_fields=['GT_CONF','GT_CONF_PERCENTILE'])


In [5]:
sample_01.list_variants_wrt(reference)

['4687t>c', '4725t>c', '13333c>z', '13335t>a', '4730_indel', '4735_indel']

In [9]:
df=sample_01.table_variants_wrt(reference)

In [10]:
df.columns

Index(['VARIANT', 'REF', 'ALT', 'GENOME_INDEX', 'GENE', 'ELEMENT_TYPE',
       'MUTATION_TYPE', 'POSITION', 'NUCLEOTIDE_NUMBER', 'AMINO_ACID_NUMBER',
       'ASSOCIATED_WITH_GENE', 'IN_PROMOTER', 'IN_CDS', 'IS_SNP', 'IS_INDEL',
       'IS_HET', 'IS_NULL', 'INDEL_LENGTH', 'INDEL_1', 'INDEL_2', 'COVERAGE',
       'HET_VARIANT_0', 'HET_VARIANT_1', 'HET_COVERAGE_0', 'HET_COVERAGE_1',
       'HET_INDEL_LENGTH_0', 'HET_INDEL_LENGTH_1', 'HET_REF', 'HET_ALT_0',
       'HET_ALT_1'],
      dtype='object')

In [13]:
df

,VARIANT,REF,ALT,GENOME_INDEX,GENE,ELEMENT_TYPE,MUTATION_TYPE,POSITION,NUCLEOTIDE_NUMBER,AMINO_ACID_NUMBER,ASSOCIATED_WITH_GENE,IN_PROMOTER,IN_CDS,IS_SNP,IS_INDEL,IS_HET,IS_NULL,INDEL_LENGTH,INDEL_1,INDEL_2,COVERAGE,HET_VARIANT_0,HET_VARIANT_1,HET_COVERAGE_0,HET_COVERAGE_1,HET_INDEL_LENGTH_0,HET_INDEL_LENGTH_1,HET_REF,HET_ALT_0,HET_ALT_1
0,t4687c,t,c,4687,M2,GENE,SNP,-37.0,-37.0,NaN,True,True,False,True,False,False,False,0,None,None,68,,,0,0,0,0,,,
1,t4725c,t,c,4725,M2,GENE,SNP,1.0,2.0,1.0,True,False,True,True,False,False,False,0,None,None,68,,,0,0,0,0,,,
2,c13333z,c,z,13333,,,SNP,NaN,NaN,NaN,True,False,False,True,False,True,False,0,None,None,119,a,g,68,51,0,0,t,a,g
3,t13335a,t,a,13335,,,SNP,NaN,NaN,NaN,True,False,False,True,False,False,False,0,None,None,68,,,0,0,0,0,,,
4,4730_indel,cg,g,4730,M2,GENE,INDEL,7.0,7.0,3.0,True,False,True,False,True,False,False,-1,4730_del,4730_del_1,68,,,0,0,0,0,,,
5,4735_indel,g,gcc,4735,M2,GENE,INDEL,12.0,12.0,4.0,True,False,True,False,True,False,False,2,4735_ins,4735_ins_2,68,,,0,0,0,0,,,


In [ ]:
for i in sample_01.genome_sequence_metadata:
    print(i)

In [ ]:
def _infer_variant_table_booleans(row):
    IS_SNP=False
    IS_INDEL=False
    IS_HET=False
    IS_NULL=False
    IN_PROMOTER=False
    IN_CDS=False
    if row["GENE"] is None:
        ASSOCIATED_WITH_GENE=False
    else:
        ASSOCIATED_WITH_GENE=True
        if row["NUCLEOTIDE_NUMBER"]<0:
            IN_PROMOTER=True
            IN_CDS=False
        elif row["NUCLEOTIDE_NUMBER"]>0:
            IN_PROMOTER=False
            IN_CDS=True
    if "indel" in row["VARIANT"]:
        IS_INDEL=True
    else:
        IS_SNP=True
    if IS_SNP:
        alt=row["VARIANT"][-1]
        if alt=="z":
            IS_HET=True
        elif alt=="x":
            IS_NULL=True
    return(pandas.Series([IS_SNP,IS_INDEL,IS_HET,IS_NULL,ASSOCIATED_WITH_GENE,IN_PROMOTER,IN_CDS]))


df[['IS_SNP','IS_INDEL','IS_HET','IS_NULL','ASSOCIATED_WITH_GENE','IN_PROMOTER','IN_CDS']]=df.apply(infer_fields,axis=1)
df

In [ ]:
m2=reference.genes['M2']

In [ ]:
m2.index[:20]

In [ ]:
m2.sequence[:20]

In [ ]:
len(m2.sequence)

In [ ]:
len(m2.index)

In [ ]:
m2.cds_index_start

In [ ]:
m2.codons